In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None

#DataFrame Worksheet - Building the Dataset
##Step 1:  
We have two lists of URLs.  The whitelist contains 400 URLs that are not malicious--specificially they are from Alexa's top 1 million websites, and another list of malicious URLs.  The first step is to read in both the whitelist and blacklists into sepearate DataFrames and take a random sample of these URLs.  The file names are ```url-whitelist.csv``` and ```url-blacklist.csv``` respectively. Next, add a column called ```isMalicious``` to both DataFrames.  Set this column to 1 for the blacklist and 0 for the whitelist.  Please drop any extraneous columns.

In [3]:
whitelist = pd.read_csv( '../Data/url-whitelist.csv', names=['rank', 'url'])
whitelist = whitelist.drop( 'rank', axis=1)
whitelist['isMalicious'] = 0

In [18]:
whitelist.head()

,url,isMalicious
0,google.com,0
1,facebook.com,0
2,youtube.com,0
3,baidu.com,0
4,yahoo.com,0


In [25]:
blacklist = pd.read_csv( '../Data/url-blacklist', names=['url','something'] )
blacklist = blacklist.drop( 'something', axis=1)
blacklist = blacklist.sample(400)
blacklist['isMalicious'] = 1

##Step 2:  Merging the data files:
Now that you have these two data sets, merge them together and create a new dataframe called ```urlData```.  If you ran ```urlData.isMalicious.value_counts()``` you should get the following output.
```python
1    400
0    400
dtype: int64
```
You should have a file called ```domainData.xlsx``` which contains whois information about the hostnames. Merge this data with the ```urlData``` dataframe.


In [27]:
urlData.isMalicious.value_counts()

1    400
0    400
dtype: int64

In [36]:
urls = pd.read_excel( '../Data/domainData.xlsx')

In [37]:
urls.head()

,url,creation_date,expiration_date,last_updated,name_servers,registrar
0,nuteczki.com,2014-04-10,2016-04-10,2015-04-03 00:00:00,"{'ns1.castpol.pl', 'ns2.castpol.pl'}",KEY-SYSTEMS GMBH
1,daftarcaramembuat.com,2015-01-31,2016-01-31,2015-05-14 00:00:00,"{'ns1.idwebhost.id', 'ns2.idwebhost.id'}",CV. JOGJACAMP
2,price59.ru,2009-06-11,2016-06-11,NaT,"{'ns4.linode.com', 'ns2.linode.com', 'ns3.lino...",REGRU-RU
3,bulinews.de,NaT,NaT,2014-11-20 11:51:55,"{'ns-de.1and1-dns.de', 'ns-de.1and1-dns.com', ...",NaN
4,buildyourownclone.com,2004-11-04,2015-11-04,2014-02-05 00:00:00,"{'yns2.yahoo.com', 'yns1.yahoo.com'}","MELBOURNE IT, LTD. D/B/A INTERNET NAMES WORLDWIDE"


##Step 2a:  Retrieving Data From WHOIS (Optional Step)
Our original idea was to have you retrieve the whois data from actual sources, however we couldn't find a free, reliable API for whois.  In any event, here is some code to do that if you want.

```python
def getUrlData( row ):
    try:
        data = pd.read_json( 'https://whois.apitruck.com/:' + row['url'] )
        data = data.drop( 'error', axis=1 )
        data = data.T
        row['created'] = data['created'].iloc[0]
        row['changed'] = data['changed'].iloc[0]
        row['expires'] = data['expires'].iloc[0]
    
        row['dnssec'] = data['dnssec'].iloc[0]
        return row
    except:
        print( "Could not find: " + row['url'] )
        row['created'] = False
        row['changed'] = False
        row['expires'] = False
        return row


urlData = urlData.apply( getUrlData, axis=1, reduce=False )
```

#Step 3:  Convert the Dates
Once you've done all that, convert all the date/time columns from strings to date/time indexes. HINT: You might want to use the ```pd.to_datetime()``` method for this.  HINT: Read the documentation for the ```coerce``` option.  Finally add a column called ```daysToExpire``` which is the difference between the creation date and the expiration date.

##Step 4:  Save your Data
The last step is to save your dataset into a CSV file by using the ```<data>.to_csv()``` function.